# Bayesian Statistic A/B Testing

This dataset comes from https://www.kaggle.com/datasets/farhadzeynalli/online-advertising-effectiveness-study-ab-testing
The exploratory analysis can be found in the AB_Test_EDA.ipynb file

We are starting with 20,000 users who were either exposed to the PSA or Ads in a 60:40 split. 
The data were collected for the frequencies, days within a month, and times of day, and number of ads the users were exposed to. 
The result is whether they made a purchase or not, though there is currently no conversion rate calculated. 

Before doing this, we should determine a priors for each group.
Since the data provided didn't include any insight as to what the product is or what industry it is involved in, the cross-market conversion rate for products is somewhere between 3 and 10 percent. Without any information, I want to ensure that I'm using a weak prior, so as not to strongly influence the outcomes, but at the same time I want to used a somewhat informed prior, since 50% conversion seems highly unlikely, especially if the company is launching an ad campaign to further drive sales. 

At this time, I am going to use Beta(1,9), setting the mean at 0.1, and a wide range of possible alternatives. 